In [1]:
print("hello")

hello


In [3]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2

cap = cv2.VideoCapture('AI Assignment video.mp4')


In [2]:
import cv2
import numpy as np

def nothing(x):
    pass

cv2.namedWindow('Trackbars')
cv2.createTrackbar('H Lower', 'Trackbars', 0, 179, nothing)
cv2.createTrackbar('S Lower', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('V Lower', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('H Upper', 'Trackbars', 179, 179, nothing)
cv2.createTrackbar('S Upper', 'Trackbars', 255, 255, nothing)
cv2.createTrackbar('V Upper', 'Trackbars', 255, 255, nothing)

def define_quadrants(frame_width, frame_height):
    quadrants = [
        ((0, 0), (frame_width // 2, frame_height // 2)),
        ((frame_width // 2, 0), (frame_width, frame_height // 2)),
        ((0, frame_height // 2), (frame_width // 2, frame_height)),
        ((frame_width // 2, frame_height // 2), (frame_width, frame_height))
    ]
    return quadrants

def detect_color(frame, color_lower, color_upper):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, color_lower, color_upper)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def get_ball_position(contours):
    if len(contours) > 0:
        c = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(c)
        if area > 50:
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            return (int(x), int(y))
    return None

def get_quadrant(position, quadrants):
    for i, ((x1, y1), (x2, y2)) in enumerate(quadrants):
        if x1 <= position[0] <= x2 and y1 <= position[1] <= y2:
            return i + 1
    return None

def draw_ball_status(frame, ball_positions, colors, ball_history, fps):
    y_offset = 30
    for i, (color, position) in enumerate(ball_positions.items()):
        status = f"{color.capitalize()} ball: {'In' if position else 'Out'}"
        if ball_history[color]:
            arrival_time = calculate_time(ball_history[color][0][1], fps)
            status += f" (Arrival: {arrival_time})"
        cv2.putText(frame, status, (10, y_offset * (i + 1)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, colors[color], 2)

def calculate_time(frame_number, fps):
    total_seconds = frame_number / fps
    minutes = int(total_seconds // 60)
    seconds = int(total_seconds % 60)
    return f"{minutes:02}:{seconds:02}"

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

print(f"Frame Width: {frame_width}")
print(f"Frame Height: {frame_height}")
print(f"FPS: {fps}")

out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
quadrants = define_quadrants(frame_width, frame_height)

colors = {
    'orange': (0, 165, 255),
    'white': (255, 255, 255),
    'blue': (255, 0, 0),
    'yellow': (0, 255, 255)
}

ball_positions = {'orange': None, 'white': None, 'blue': None, 'yellow': None}
ball_history = {'orange': [], 'white': [], 'blue': [], 'yellow': []}

color_ranges = {
    'orange': ((5, 100, 100), (15, 255, 255)),
    'white': ((0, 0, 200), (180, 20, 255)),
    'blue': ((100, 150, 0), (140, 255, 255)),
    'yellow': ((20, 100, 100), (30, 255, 255))
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    for color in ['orange', 'white', 'blue', 'yellow']:
        contours = detect_color(frame, color_ranges[color][0], color_ranges[color][1])
        position = get_ball_position(contours)
        if position:
            ball_positions[color] = position
            quadrant = get_quadrant(position, quadrants)
            frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            time_str = calculate_time(frame_number, fps)
            if len(ball_history[color]) == 0 or ball_history[color][-1][0] != quadrant:
                ball_history[color].append((quadrant, frame_number, time_str))
            cv2.circle(frame, position, 10, colors[color], -1)
        else:
            ball_positions[color] = None

    for ((x1, y1), (x2, y2)) in quadrants:
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)

    draw_ball_status(frame, ball_positions, colors, ball_history, fps)

    out.write(frame)

    frames = cv2.resize(frame, (720, 360))
    cv2.imshow('Frame', frames)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

for color, history in ball_history.items():
    print(f"\n{color.capitalize()} ball history:")
    for entry in history:
        print(f"Quadrant: {entry[0]}, Frame: {entry[1]}, Time: {entry[2]}")


Frame Width: 1920
Frame Height: 1080
FPS: 30.04340861961722

Orange ball history:
Quadrant: 4, Frame: 61, Time: 00:02
Quadrant: 2, Frame: 64, Time: 00:02
Quadrant: 4, Frame: 69, Time: 00:02
Quadrant: 2, Frame: 282, Time: 00:09
Quadrant: 4, Frame: 362, Time: 00:12
Quadrant: 2, Frame: 705, Time: 00:23
Quadrant: 4, Frame: 707, Time: 00:23
Quadrant: 2, Frame: 708, Time: 00:23
Quadrant: 4, Frame: 738, Time: 00:24
Quadrant: 3, Frame: 818, Time: 00:27
Quadrant: 4, Frame: 823, Time: 00:27

White ball history:
Quadrant: 2, Frame: 1, Time: 00:00

Blue ball history:
Quadrant: 4, Frame: 1, Time: 00:00
Quadrant: 2, Frame: 53, Time: 00:01
Quadrant: 4, Frame: 54, Time: 00:01
Quadrant: 2, Frame: 55, Time: 00:01
Quadrant: 4, Frame: 56, Time: 00:01
Quadrant: 2, Frame: 62, Time: 00:02
Quadrant: 4, Frame: 70, Time: 00:02
Quadrant: 2, Frame: 71, Time: 00:02
Quadrant: 4, Frame: 74, Time: 00:02
Quadrant: 2, Frame: 77, Time: 00:02
Quadrant: 4, Frame: 80, Time: 00:02
Quadrant: 2, Frame: 84, Time: 00:02
Quadran